In [486]:
import os
# import pandas as pd 
import polars as pl
# import scanpy as sc
import numpy as np
import pandas as pd
import typing as tp
import re

In [487]:
statarg = 'single'
OutputDir = f'./data/2_PCAUMAP'
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)
feat_base = './data/1_FeaturesImages_meanstd_PerPlate'
feat_agg = '/share/data/analyses/christa/colopaint3D/spher_colo52_v1'

In [488]:

filenames = ['selected_data_all']

statmet = 'WellAggregates'

statmets = ['SingleCell', 'Aggregates']

dropcols = ['Nuclei_Location_Center_X',
            'Nuclei_Location_Center_Y',
            'Location',
            'ImageNumber_',
            'Parent',
            'Children',
            '_ObjectNumber',
            '_Object_Number',
            '_Y',
            '_X' 
            ]


cons = ['pos_con', 'neg_con']
nodmso = ['trt']
wdmso = ['pos_con']
float_columns=[pl.col(pl.Float32),pl.col(pl.Float64)]
str_columns = [pl.col(pl.String)]
str_columns = [pl.col(pl.Int64),pl.col(pl.Int32)]
feat_sc = f'{feat_base}/{statmet}'


In [489]:
def readData(filename, feat_dir, statmet='SingleCell', filetype='parquet'):
    if filetype=='parquet':
        df = pl.read_parquet(f'{feat_dir}/{filename}.parquet')
    elif filetype=='csv':
        df = pl.read_csv(f'{feat_dir}/{filename}.csv')
    ##This here is important!!!!!!
    # df=df.select([c for c in df.columns if not is_meta_column(c)])
    # dataNpy = df.to_numpy(df.select(float_columns))
    ##
    onehot_list = df.select(['Metadata_pathway'])['Metadata_pathway'].unique().to_list()
    onehot_mapping = {name: i for i, name in enumerate(onehot_list)}
    df = df.with_columns(df['Metadata_pathway'].map_elements(lambda name: oneHot(name, onehot_mapping)).alias('Metadata_cmpd_onehot'))
    return df

# this is code from Dan
def is_meta_column(
    c:str,
    allowlist:tp.List[str]=["Metadata_AcqID","Metadata_Site"],
    denylist:tp.List[str]=["Metadata_Well","Metadata_barcode"],
)->bool:
    """
        allowlist:
            the function will return False for these, no matter if they are metadata or not
        denylist:
            the function will return True for these, no matter if they are metadata or not
    """
    if c in allowlist:
        return False
    if c in denylist:
        return True
    for ex in '''
        Metadata
        Plate
        Well
    '''.split():
        if re.search(ex, c):
            return True
    return False


def oneHot(row, mapping):
    return mapping.get(row, -1)


def filter_and_clean(df):
    df_3d = scrub(df, type='aggregates')
    # print(df_3d.head(), df.head())
    df_2d = scrub(df,'2D')
    df_mip = scrub(df,'MIP')
    return df_3d, df_2d, df_mip

def scrub(df, type):
    df = df.filter(pl.col('Metadata_data_type')==type)
    df_floats=df.select([c for c in df.columns if not is_meta_column(c)]).select(float_columns)
    cols_to_drop = [col for col in df_floats.columns if df_floats.select(pl.col(col).is_nan().all())]
    df = df.drop(cols_to_drop)
    for col in df.select([pl.col(pl.Float32),pl.col(pl.Float64)]).columns:
        df=df.filter(pl.col(col).is_not_null())
    return df


In [490]:
df = pl.read_parquet(f'{feat_agg}/{filenames[0]}_cleaned.parquet')
# df = pd.read_csv(f'{feat_agg}/{filenames[0]}.csv', sep=',')
# df = df.fillna(np.nan)
# df = pl.from_pandas(df)
# df.write_csv(f'{feat_agg}/{filenames[0]}_cleaned.csv', separator=',')
# df.write_parquet(f'{feat_agg}/{filenames[0]}_cleaned.parquet')

In [491]:
# df.write_csv(f'{feat_agg}/{filenames[0]}_cleaned.csv', separator=',')
# df.write_parquet(f'{feat_agg}/{filenames[0]}_cleaned.parquet')

In [492]:
df.head()

Metadata_Well,Metadata_barcode,Metadata_PlateWell,Metadata_cell_line,Metadata_cmpd_conc,Metadata_cmpdname,Metadata_name,Metadata_pert_type,Metadata_pathway,Metadata_target,Nuclei_Granularity_3_CONC,Cells_RadialDistribution_ZernikePhase_MITO_6_6,Cytoplasm_AreaShape_Zernike_9_3,Cells_Correlation_Correlation_CONC_SYTO,Nuclei_RadialDistribution_ZernikePhase_MITO_9_3,Cells_Granularity_2_HOECHST,Cells_RadialDistribution_ZernikePhase_PHAandWGA_7_5,Nuclei_RadialDistribution_ZernikePhase_PHAandWGA_3_3,Nuclei_Correlation_K_HOECHST_SYTO,Nuclei_RadialDistribution_ZernikePhase_MITO_7_1,Cells_Correlation_K_SYTO_PHAandWGA,Cells_Correlation_Correlation_HOECHST_SYTO,Cells_Intensity_MassDisplacement_MITO,Cells_RadialDistribution_ZernikePhase_CONC_9_7,Nuclei_RadialDistribution_ZernikePhase_PHAandWGA_9_3,Cells_RadialDistribution_ZernikePhase_HOECHST_4_2,Nuclei_Granularity_13_CONC,Cells_RadialDistribution_MeanFrac_MITO_1of4,Cytoplasm_Granularity_2_HOECHST,Cytoplasm_AreaShape_Zernike_6_4,Cytoplasm_RadialDistribution_ZernikePhase_SYTO_7_1,Cells_RadialDistribution_ZernikePhase_CONC_1_1,Cytoplasm_AreaShape_Extent,Cytoplasm_RadialDistribution_ZernikePhase_CONC_9_7,Cells_AreaShape_Compactness,Nuclei_RadialDistribution_FracAtD_SYTO_1of4,Nuclei_Granularity_12_MITO,…,RadialDistribution_ZernikeMagnitude_PHAandWGA_7_5,Correlation_RWC_SYTO_CONC,Intensity_StdIntensity_HOECHST,Intensity_UpperQuartileIntensity_MITO,RadialDistribution_RadialCV_CONC_1of4,Granularity_2_PHAandWGA,Location_MaxIntensity_X_SYTO,RadialDistribution_ZernikeMagnitude_HOECHST_4_0,RadialDistribution_ZernikeMagnitude_MITO_4_2,RadialDistribution_ZernikeMagnitude_MITO_2_0,Granularity_10_CONC,RadialDistribution_ZernikePhase_PHAandWGA_8_2,RadialDistribution_ZernikeMagnitude_HOECHST_7_5,Correlation_Manders_HOECHST_MITO,Intensity_MADIntensity_HOECHST,Correlation_K_PHAandWGA_SYTO,Location_MaxIntensity_Y_HOECHST,RadialDistribution_ZernikeMagnitude_MITO_9_7,RadialDistribution_ZernikePhase_MITO_5_1,RadialDistribution_ZernikePhase_PHAandWGA_3_3,AreaShape_Zernike_5_1,RadialDistribution_ZernikePhase_SYTO_1_1,Granularity_14_PHAandWGA,Granularity_11_HOECHST,AreaShape_Zernike_4_2,Intensity_MADIntensity_SYTO,RadialDistribution_ZernikeMagnitude_MITO_8_0,Granularity_10_HOECHST,RadialDistribution_ZernikeMagnitude_SYTO_5_3,RadialDistribution_ZernikePhase_MITO_4_2,RadialDistribution_ZernikeMagnitude_MITO_6_0,RadialDistribution_ZernikePhase_HOECHST_1_1,RadialDistribution_ZernikeMagnitude_SYTO_4_0,RadialDistribution_ZernikeMagnitude_MITO_7_1,Correlation_Correlation_MITO_PHAandWGA,Correlation_K_SYTO_PHAandWGA,RadialDistribution_ZernikePhase_CONC_6_2
str,str,str,str,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""B02""","""PB000141""","""B02_PB000141""","""HT29""",3.0,"""PD0325901""","""PD032""","""trt""","""MAPK""","""MEK""",-2.086016,null,-1.128686,0.375602,0.14308,2.125792,-1.107771,-1.290878,-13.276694,0.261465,1.377393,null,0.072812,-0.600456,-0.188802,null,-0.67957,-1.341828,null,-0.86925,-0.328614,0.269362,1.962055,0.319998,-3.483608,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""B04""","""PB000141""","""B04_PB000141""","""HT29""",3.0,"""Olaparib (AZD2281, Ku-0059436)""","""Olapa""","""trt""","""DNA Damage""","""PARP""",-0.497579,null,-0.101863,0.896907,0.764776,-0.479159,0.001408,0.041524,1.936808,-0.007276,0.563188,null,1.252764,-0.497035,0.719203,null,-0.591146,0.132747,null,-0.155315,0.708237,0.067664,1.006738,-0.318597,-0.067316,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"

In [493]:
df.select(pl.col('Metadata_data_type')).unique()

Metadata_data_type
str
"""MIP"""
"""2D"""
"""aggregates"""


In [494]:
df.select(pl.col('Metadata_cell_line')).unique()

Metadata_cell_line
str
"""HT29"""
"""HCT116"""


In [495]:
df_hct = df.filter(pl.col('Metadata_cell_line')=='HCT116')
df_ht = df.filter(pl.col('Metadata_cell_line')=='HT29')


In [496]:
df1 = df_hct.filter(pl.col('Metadata_data_type')=='2D')

In [497]:
df1.shape

(1185, 2357)

In [498]:
df_floats=df1.select([c for c in df1.columns if not is_meta_column(c)]).select(float_columns)

In [499]:
df_floats.shape

(1185, 2346)

In [500]:
# df_bool = df_floats.select(pl.all().is_nan().all())

df_bool = df_floats.select(pl.all().is_nan().all()) 

cols_to_drop = [col for col, is_all_nan in zip(df_floats.columns, df_bool.row(0)) if is_all_nan]


In [501]:
df_bool

Nuclei_Granularity_3_CONC,Cells_RadialDistribution_ZernikePhase_MITO_6_6,Cytoplasm_AreaShape_Zernike_9_3,Cells_Correlation_Correlation_CONC_SYTO,Nuclei_RadialDistribution_ZernikePhase_MITO_9_3,Cells_Granularity_2_HOECHST,Cells_RadialDistribution_ZernikePhase_PHAandWGA_7_5,Nuclei_RadialDistribution_ZernikePhase_PHAandWGA_3_3,Nuclei_Correlation_K_HOECHST_SYTO,Nuclei_RadialDistribution_ZernikePhase_MITO_7_1,Cells_Correlation_K_SYTO_PHAandWGA,Cells_Correlation_Correlation_HOECHST_SYTO,Cells_Intensity_MassDisplacement_MITO,Cells_RadialDistribution_ZernikePhase_CONC_9_7,Nuclei_RadialDistribution_ZernikePhase_PHAandWGA_9_3,Cells_RadialDistribution_ZernikePhase_HOECHST_4_2,Nuclei_Granularity_13_CONC,Cells_RadialDistribution_MeanFrac_MITO_1of4,Cytoplasm_Granularity_2_HOECHST,Cytoplasm_AreaShape_Zernike_6_4,Cytoplasm_RadialDistribution_ZernikePhase_SYTO_7_1,Cells_RadialDistribution_ZernikePhase_CONC_1_1,Cytoplasm_AreaShape_Extent,Cytoplasm_RadialDistribution_ZernikePhase_CONC_9_7,Cells_AreaShape_Compactness,Nuclei_RadialDistribution_FracAtD_SYTO_1of4,Nuclei_Granularity_12_MITO,Nuclei_Correlation_K_CONC_SYTO,Cells_RadialDistribution_RadialCV_SYTO_2of4,Nuclei_Correlation_RWC_HOECHST_SYTO,Cells_Granularity_2_SYTO,Cytoplasm_Granularity_16_CONC,Nuclei_Granularity_3_HOECHST,Nuclei_RadialDistribution_ZernikePhase_MITO_7_5,Cytoplasm_RadialDistribution_ZernikePhase_SYTO_9_7,Cells_RadialDistribution_ZernikeMagnitude_MITO_6_4,Nuclei_RadialDistribution_ZernikePhase_CONC_1_1,…,RadialDistribution_ZernikeMagnitude_PHAandWGA_7_5,Correlation_RWC_SYTO_CONC,Intensity_StdIntensity_HOECHST,Intensity_UpperQuartileIntensity_MITO,RadialDistribution_RadialCV_CONC_1of4,Granularity_2_PHAandWGA,Location_MaxIntensity_X_SYTO,RadialDistribution_ZernikeMagnitude_HOECHST_4_0,RadialDistribution_ZernikeMagnitude_MITO_4_2,RadialDistribution_ZernikeMagnitude_MITO_2_0,Granularity_10_CONC,RadialDistribution_ZernikePhase_PHAandWGA_8_2,RadialDistribution_ZernikeMagnitude_HOECHST_7_5,Correlation_Manders_HOECHST_MITO,Intensity_MADIntensity_HOECHST,Correlation_K_PHAandWGA_SYTO,Location_MaxIntensity_Y_HOECHST,RadialDistribution_ZernikeMagnitude_MITO_9_7,RadialDistribution_ZernikePhase_MITO_5_1,RadialDistribution_ZernikePhase_PHAandWGA_3_3,AreaShape_Zernike_5_1,RadialDistribution_ZernikePhase_SYTO_1_1,Granularity_14_PHAandWGA,Granularity_11_HOECHST,AreaShape_Zernike_4_2,Intensity_MADIntensity_SYTO,RadialDistribution_ZernikeMagnitude_MITO_8_0,Granularity_10_HOECHST,RadialDistribution_ZernikeMagnitude_SYTO_5_3,RadialDistribution_ZernikePhase_MITO_4_2,RadialDistribution_ZernikeMagnitude_MITO_6_0,RadialDistribution_ZernikePhase_HOECHST_1_1,RadialDistribution_ZernikeMagnitude_SYTO_4_0,RadialDistribution_ZernikeMagnitude_MITO_7_1,Correlation_Correlation_MITO_PHAandWGA,Correlation_K_SYTO_PHAandWGA,RadialDistribution_ZernikePhase_CONC_6_2
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,…,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,…,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true


In [502]:
# df_bool_2 = df_floats.select(pl.all().is_null().all())

# cols_to_drop_2 = [col for col, is_all_nan in zip(df_floats.columns, df_bool_2.row(0)) if is_all_nan]

In [503]:
df1 = df1.drop(cols_to_drop)

In [504]:
df1.head()

Metadata_Well,Metadata_barcode,Metadata_PlateWell,Metadata_cell_line,Metadata_cmpd_conc,Metadata_cmpdname,Metadata_name,Metadata_pert_type,Metadata_pathway,Metadata_target,Metadata_data_type,Intensity_StdIntensity_illumSYTO_nuclei,RadialDistribution_ZernikePhase_illumPHAandWGA_5_5_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_1_1_cytoplasm,Correlation_K_illumSYTO_illumCONC_nuclei,RadialDistribution_ZernikeMagnitude_illumSYTO_9_7_cells,Correlation_Overlap_illumPHAandWGA_illumSYTO_cytoplasm,RadialDistribution_ZernikePhase_illumCONC_6_4_nuclei,RadialDistribution_ZernikePhase_illumCONC_5_5_cytoplasm,RadialDistribution_ZernikeMagnitude_illumPHAandWGA_2_2_nuclei,Intensity_UpperQuartileIntensity_illumSYTO_nuclei,RadialDistribution_ZernikePhase_illumSYTO_7_7_nuclei,RadialDistribution_ZernikePhase_illumHOECHST_9_9_nuclei,RadialDistribution_ZernikePhase_illumMITO_5_1_nuclei,RadialDistribution_ZernikePhase_illumCONC_1_1_cells,Correlation_RWC_illumHOECHST_illumPHAandWGA_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_8_6_cells,RadialDistribution_ZernikePhase_illumPHAandWGA_9_1_cells,AreaShape_Zernike_9_1_cytoplasm,AreaShape_Zernike_7_1_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_3_1_nuclei,Granularity_3_illumPHAandWGA_cells,Granularity_3_illumMITO_cells,RadialDistribution_ZernikeMagnitude_illumHOECHST_4_0_nuclei,Granularity_4_illumPHAandWGA_cytoplasm,RadialDistribution_ZernikeMagnitude_illumSYTO_2_0_cytoplasm,Correlation_RWC_illumMITO_illumSYTO_cells,…,RadialDistribution_ZernikePhase_illumCONC_8_8_cytoplasm,Correlation_K_illumHOECHST_illumCONC_cells,RadialDistribution_MeanFrac_illumHOECHST_1of4_cytoplasm,Granularity_11_illumCONC_nuclei,AreaShape_Zernike_9_5_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_3_3_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_7_1_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_7_3_cytoplasm,RadialDistribution_ZernikePhase_illumMITO_6_2_nuclei,RadialDistribution_RadialCV_illumMITO_1of4_cells,Granularity_13_illumHOECHST_cells,RadialDistribution_RadialCV_illumHOECHST_2of4_nuclei,Correlation_Manders_illumMITO_illumSYTO_cytoplasm,Correlation_K_illumHOECHST_illumSYTO_nuclei,RadialDistribution_ZernikeMagnitude_illumHOECHST_8_6_nuclei,RadialDistribution_ZernikeMagnitude_illumCONC_8_0_cells,RadialDistribution_ZernikePhase_illumPHAandWGA_9_5_nuclei,RadialDistribution_ZernikePhase_illumCONC_7_7_nuclei,RadialDistribution_ZernikePhase_illumHOECHST_4_4_nuclei,RadialDistribution_ZernikeMagnitude_illumPHAandWGA_4_2_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_9_5_cells,RadialDistribution_ZernikeMagnitude_illumHOECHST_2_0_cytoplasm,RadialDistribution_ZernikePhase_illumCONC_5_3_cytoplasm,RadialDistribution_ZernikePhase_illumCONC_7_5_nuclei,RadialDistribution_ZernikePhase_illumMITO_8_8_cells,RadialDistribution_ZernikeMagnitude_illumCONC_1_1_cytoplasm,AreaShape_Zernike_4_4_nuclei,Correlation_Correlation_illumHOECHST_illumSYTO_nuclei,RadialDistribution_ZernikeMagnitude_illumPHAandWGA_4_0_nuclei,Intensity_MaxIntensity_illumCONC_cytoplasm,Correlation_Correlation_illumHOECHST_illumMITO_cytoplasm,RadialDistribution_FracAtD_illumHOECHST_3of4_cytoplasm,RadialDistribution_ZernikeMagnitude_illumPHAandWGA_5_1_cytoplasm,RadialDistribution_ZernikeMagnitude_illumSYTO_6_0_nuclei,RadialDistribution_ZernikePhase_illumMITO_4_4_nuclei,RadialDistribution_ZernikePhase_illumHOECHST_2_2_cells,RadialDistribution_ZernikePhase_illumCONC_3_1_nuclei
str,str,str,str,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""B02""","""PB000036""","""B02_PB000036""","""HCT116""",3.0,"""flup""","""flup""","""pos_con""",null,null,"""2D""",0.20286,-0.568621,-0.303259,-0.228477,0.486931,-2.911519,0.740986,0.461446,1.63652,-0.318561,-0.329387,1.868213,1.603759,-0.150777,0.952731,0.320654

In [505]:
df_floats=df1.select([c for c in df1.columns if not is_meta_column(c)]).select(float_columns)

In [506]:
has_nan = df_floats.select(pl.all().is_nan().any()).row(0)[0]

In [507]:
has_nan

False

In [508]:
df1.shape

(1185, 860)

In [509]:
df1 = df1.with_columns(
    pl.when(pl.col('Metadata_pathway').is_null())
      .then(pl.col('Metadata_cmpdname'))
      .otherwise(pl.col('Metadata_pathway'))
      .alias('Metadata_pathway')
)

In [510]:
l1 = df1.select('Metadata_pathway').unique().to_series().to_list()

In [511]:
len(l1)


22

In [512]:
l1

['JAK/STAT',
 'fenb',
 'Cytoskeletal Signaling',
 'flup',
 'Protein Tyrosine Kinase',
 'MAPK',
 'Stem Cells & Wnt',
 'DNA Damage',
 'dmso',
 'Apoptosis',
 'Epigenetics',
 'Angiogenesis',
 'Others',
 'stau',
 'etop',
 'Proteases',
 'TGF-beta/Smad',
 'water',
 'Cell Cycle',
 'PI3K/Akt/mTOR',
 'sorb',
 'water+']

In [513]:
l2 = ['Cell Cycle',
 'Proteases',
 'Apoptosis',
 'Epigenetics',
 'Others',
 'stau',
 'Cytoskeletal Signaling',
 'PI3K/Akt/mTOR',
 'DNA Damage',
 'TGF-beta/Smad',
 'Protein Tyrosine Kinase',
 'fenb',
 'sorb',
 'water',
 'dmso',
 'MAPK',
 'Stem Cells & Wnt',
 'JAK/STAT',
 'Angiogenesis',
 'etop']

In [514]:
l2

['Cell Cycle',
 'Proteases',
 'Apoptosis',
 'Epigenetics',
 'Others',
 'stau',
 'Cytoskeletal Signaling',
 'PI3K/Akt/mTOR',
 'DNA Damage',
 'TGF-beta/Smad',
 'Protein Tyrosine Kinase',
 'fenb',
 'sorb',
 'water',
 'dmso',
 'MAPK',
 'Stem Cells & Wnt',
 'JAK/STAT',
 'Angiogenesis',
 'etop']

In [515]:
L3 = [l1 for l1 in l1 if l1 not in l2] + [l2 for l2 in l2 if l2 not in l1]





In [516]:
L3

['flup', 'water+']

In [517]:
ignore_list = ['water','water+','flup','dmso']

In [518]:
df1.head()

Metadata_Well,Metadata_barcode,Metadata_PlateWell,Metadata_cell_line,Metadata_cmpd_conc,Metadata_cmpdname,Metadata_name,Metadata_pert_type,Metadata_pathway,Metadata_target,Metadata_data_type,Intensity_StdIntensity_illumSYTO_nuclei,RadialDistribution_ZernikePhase_illumPHAandWGA_5_5_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_1_1_cytoplasm,Correlation_K_illumSYTO_illumCONC_nuclei,RadialDistribution_ZernikeMagnitude_illumSYTO_9_7_cells,Correlation_Overlap_illumPHAandWGA_illumSYTO_cytoplasm,RadialDistribution_ZernikePhase_illumCONC_6_4_nuclei,RadialDistribution_ZernikePhase_illumCONC_5_5_cytoplasm,RadialDistribution_ZernikeMagnitude_illumPHAandWGA_2_2_nuclei,Intensity_UpperQuartileIntensity_illumSYTO_nuclei,RadialDistribution_ZernikePhase_illumSYTO_7_7_nuclei,RadialDistribution_ZernikePhase_illumHOECHST_9_9_nuclei,RadialDistribution_ZernikePhase_illumMITO_5_1_nuclei,RadialDistribution_ZernikePhase_illumCONC_1_1_cells,Correlation_RWC_illumHOECHST_illumPHAandWGA_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_8_6_cells,RadialDistribution_ZernikePhase_illumPHAandWGA_9_1_cells,AreaShape_Zernike_9_1_cytoplasm,AreaShape_Zernike_7_1_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_3_1_nuclei,Granularity_3_illumPHAandWGA_cells,Granularity_3_illumMITO_cells,RadialDistribution_ZernikeMagnitude_illumHOECHST_4_0_nuclei,Granularity_4_illumPHAandWGA_cytoplasm,RadialDistribution_ZernikeMagnitude_illumSYTO_2_0_cytoplasm,Correlation_RWC_illumMITO_illumSYTO_cells,…,RadialDistribution_ZernikePhase_illumCONC_8_8_cytoplasm,Correlation_K_illumHOECHST_illumCONC_cells,RadialDistribution_MeanFrac_illumHOECHST_1of4_cytoplasm,Granularity_11_illumCONC_nuclei,AreaShape_Zernike_9_5_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_3_3_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_7_1_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_7_3_cytoplasm,RadialDistribution_ZernikePhase_illumMITO_6_2_nuclei,RadialDistribution_RadialCV_illumMITO_1of4_cells,Granularity_13_illumHOECHST_cells,RadialDistribution_RadialCV_illumHOECHST_2of4_nuclei,Correlation_Manders_illumMITO_illumSYTO_cytoplasm,Correlation_K_illumHOECHST_illumSYTO_nuclei,RadialDistribution_ZernikeMagnitude_illumHOECHST_8_6_nuclei,RadialDistribution_ZernikeMagnitude_illumCONC_8_0_cells,RadialDistribution_ZernikePhase_illumPHAandWGA_9_5_nuclei,RadialDistribution_ZernikePhase_illumCONC_7_7_nuclei,RadialDistribution_ZernikePhase_illumHOECHST_4_4_nuclei,RadialDistribution_ZernikeMagnitude_illumPHAandWGA_4_2_cytoplasm,RadialDistribution_ZernikePhase_illumHOECHST_9_5_cells,RadialDistribution_ZernikeMagnitude_illumHOECHST_2_0_cytoplasm,RadialDistribution_ZernikePhase_illumCONC_5_3_cytoplasm,RadialDistribution_ZernikePhase_illumCONC_7_5_nuclei,RadialDistribution_ZernikePhase_illumMITO_8_8_cells,RadialDistribution_ZernikeMagnitude_illumCONC_1_1_cytoplasm,AreaShape_Zernike_4_4_nuclei,Correlation_Correlation_illumHOECHST_illumSYTO_nuclei,RadialDistribution_ZernikeMagnitude_illumPHAandWGA_4_0_nuclei,Intensity_MaxIntensity_illumCONC_cytoplasm,Correlation_Correlation_illumHOECHST_illumMITO_cytoplasm,RadialDistribution_FracAtD_illumHOECHST_3of4_cytoplasm,RadialDistribution_ZernikeMagnitude_illumPHAandWGA_5_1_cytoplasm,RadialDistribution_ZernikeMagnitude_illumSYTO_6_0_nuclei,RadialDistribution_ZernikePhase_illumMITO_4_4_nuclei,RadialDistribution_ZernikePhase_illumHOECHST_2_2_cells,RadialDistribution_ZernikePhase_illumCONC_3_1_nuclei
str,str,str,str,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""B02""","""PB000036""","""B02_PB000036""","""HCT116""",3.0,"""flup""","""flup""","""pos_con""","""flup""",null,"""2D""",0.20286,-0.568621,-0.303259,-0.228477,0.486931,-2.911519,0.740986,0.461446,1.63652,-0.318561,-0.329387,1.868213,1.603759,-0.150777,0.952731,0.